In [1]:
import warnings
warnings.filterwarnings("ignore") 

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import timedelta,datetime
df=pd.read_csv('C:\\workplace\\electricity_demand\\dataset\\AEP_hourly.csv')
df.columns=['date_time','aep_mw']
df['date_time']=pd.to_datetime(df['date_time'])
df.sort_values('date_time',ascending=True,inplace=True)
df['date_time']=df['date_time'].astype(str)

In [3]:
def preparion_train (df,num_lags,lag_feature):

    df_lags=pd.DataFrame(index=df.index,columns=['lags_{}'.format(671-i) for i in range(num_lags)])
    for i in range(num_lags):
        df_lags['lags_{}'.format(671-i)]=df[lag_feature].shift(671-i)
    
    df_targets=pd.DataFrame(index=df.index,columns=['step_{}'.format(i) for i in range(16,40)])
    for i in range(16,40):
        df_targets['step_{}'.format(i)]=df[lag_feature].shift(-i)
    
    df_processed=pd.concat([df[['date_time']],df_targets,df_lags],axis=1)
    #df_processed=df_processed[pd.to_datetime(df_processed['date_time']).dt.hour==trigerred_time]
    return df_processed.dropna()

# Train and Test split

In [4]:
from sklearn.preprocessing import PowerTransformer
scaler=PowerTransformer().fit(df.loc[df['date_time']<='2017-08-02 23:00:00',['aep_mw']])
df['scaled_aep_mw']=pd.Series(scaler.transform(df[['aep_mw']]).reshape(-1),index=df.index)
df.head()

,date_time,aep_mw,scaled_aep_mw
2183,2004-10-01 01:00:00,12379.0,-1.276695
2184,2004-10-01 02:00:00,11935.0,-1.492740
2185,2004-10-01 03:00:00,11692.0,-1.614218
2186,2004-10-01 04:00:00,11597.0,-1.662359
2187,2004-10-01 05:00:00,11681.0,-1.619773


In [5]:
df_processed=preparion_train (df,24*7*4,'scaled_aep_mw')
df_processed

,date_time,step_16,step_17,step_18,step_19,step_20,step_21,step_22,step_23,step_24,...,lags_9,lags_8,lags_7,lags_6,lags_5,lags_4,lags_3,lags_2,lags_1,lags_0
1558,2004-10-29 00:00:00,-0.105342,-0.080372,-0.156780,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,...,-0.030725,-0.076813,-0.122441,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436
1511,2004-10-29 01:00:00,-0.080372,-0.156780,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,...,-0.076813,-0.122441,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351
1512,2004-10-29 02:00:00,-0.156780,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,-1.622300,...,-0.122441,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351,-1.449815
1513,2004-10-29 03:00:00,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,-1.622300,-1.816869,...,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351,-1.449815,-1.583501
1514,2004-10-29 04:00:00,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,-1.622300,-1.816869,-1.954480,...,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351,-1.449815,-1.583501,-1.696536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116166,2018-08-01 05:00:00,0.422499,0.326766,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,...,0.501312,0.458576,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001
116167,2018-08-01 06:00:00,0.326766,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,...,0.458576,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482
116168,2018-08-01 07:00:00,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,-0.691662,...,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482,-0.756927
116169,2018-08-01 08:00:00,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,-0.691662,-0.408344,...,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482,-0.756927,-0.486737


In [6]:
X_train=df_processed.loc[df_processed['date_time']<='2017-08-02 23:00:00',df_processed.columns[df_processed.columns.str.contains('lags_')]]
X_test=df_processed.loc[(df_processed['date_time']>='2017-08-02 08:00:00') &\
    (pd.to_datetime(df_processed['date_time']).dt.hour==8),df_processed.columns[df_processed.columns.str.contains('lags_')]]
X_train=np.array(X_train).reshape(X_train.shape[0],1,X_train.shape[1])
X_test=np.array(X_test).reshape(X_test.shape[0],1,X_test.shape[1])

y_train=df_processed.loc[df_processed['date_time']<='2017-08-02 23:00:00',df_processed.columns[df_processed.columns.str.contains('step_')]]
y_test=df_processed.loc[(df_processed['date_time']>='2017-08-02 08:00:00') &\
    (pd.to_datetime(df_processed['date_time']).dt.hour==8),df_processed.columns[df_processed.columns.str.contains('step_')]]

# Hyperparameter tuning

In [7]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Input,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
from sklearn.metrics import mean_absolute_percentage_error

In [9]:
import optuna

In [10]:
from sklearn.model_selection import train_test_split 
X_fit,X_val,y_fit,y_val=train_test_split(X_train,y_train,test_size=0.3,shuffle=False)

In [14]:
stoppage_rule=EarlyStopping(patience=10,monitor='val_loss')

In [15]:
def val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s):

    # Input layer
    model=Sequential()
    #model.add(Input(X_fit.shape[0],X_fit.shape[1],X_fit.shape[2]))
    # Hidden Layers
    for i in range(num_layers):
        if i==0:
            model.add(LSTM(units=num_nerons,return_sequences=True,input_shape=(X_fit.shape[1],X_fit.shape[2])))
        elif i<num_layers-1:
            model.add(LSTM(units=num_nerons,return_sequences=True))
        else :
            model.add(LSTM(units=num_nerons))
    # output layer
    model.add(Dense(units=24))
    # Training
    model.compile(optimizer=Adam(learning_rate=alpha),loss='mse')
    model.fit(X_fit,y_fit,validation_data=(X_val,y_val),epochs=iter,batch_size=b_s,callbacks=[stoppage_rule])
    # Evaluation of the model
    pred=model.predict(X_val)
    y_val2=scaler.inverse_transform(np.array(y_val).reshape(-1,1))
    pred2=scaler.inverse_transform(pred.reshape(-1,1))
    error=mean_absolute_percentage_error(y_val2,pred2)
    return error

In [16]:
def objective(trial):

    num_layers=trial.suggest_int('num_layers',1,4)
    num_nerons=trial.suggest_int('num_nerons',32,256,32)
    alpha=trial.suggest_float('alpha',0.001,0.1)
    iter=trial.suggest_int('iter',1000,1000)
    b_s=trial.suggest_int('b_s',32,1024,32)
    loss_fun=val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s)
    return loss_fun 
study = optuna.create_study()
study.optimize(objective,n_trials=100)

[I 2024-02-17 13:28:01,075] A new study created in memory with name: no-name-53076768-ebc0-486e-8164-18a186ed48c8


Epoch 1/1000
85/85 [==============================] - 19s 158ms/step - loss: 0.9602 - val_loss: 1.1910
Epoch 2/1000
85/85 [==============================] - 12s 141ms/step - loss: 0.9516 - val_loss: 1.1743
Epoch 3/1000
85/85 [==============================] - 12s 140ms/step - loss: 0.9510 - val_loss: 1.1618
Epoch 4/1000
85/85 [==============================] - 12s 139ms/step - loss: 0.9513 - val_loss: 1.1682
Epoch 5/1000
85/85 [==============================] - 12s 138ms/step - loss: 0.9509 - val_loss: 1.1594
Epoch 6/1000
85/85 [==============================] - 12s 142ms/step - loss: 0.9511 - val_loss: 1.1659
Epoch 7/1000
85/85 [==============================] - 11s 131ms/step - loss: 0.9509 - val_loss: 1.1603
Epoch 8/1000
85/85 [==============================] - 10s 114ms/step - loss: 0.9510 - val_loss: 1.1557
Epoch 9/1000
85/85 [==============================] - 10s 114ms/step - loss: 0.9510 - val_loss: 1.1851
Epoch 10/1000
85/85 [==============================] - 10s 116ms/step - l

[I 2024-02-17 13:32:01,581] Trial 0 finished with value: 0.1565835162874604 and parameters: {'num_layers': 1, 'num_nerons': 32, 'alpha': 0.05137421313006922, 'iter': 1000, 'b_s': 928}. Best is trial 0 with value: 0.1565835162874604.


Epoch 1/1000
306/306 [==============================] - 19s 34ms/step - loss: 0.9639 - val_loss: 1.1786
Epoch 2/1000
306/306 [==============================] - 9s 29ms/step - loss: 0.9519 - val_loss: 1.1837
Epoch 3/1000
306/306 [==============================] - 9s 29ms/step - loss: 0.9519 - val_loss: 1.1729
Epoch 4/1000
306/306 [==============================] - 9s 29ms/step - loss: 0.9519 - val_loss: 1.1774
Epoch 5/1000
306/306 [==============================] - 10s 33ms/step - loss: 0.9514 - val_loss: 1.2330
Epoch 6/1000
306/306 [==============================] - 8s 26ms/step - loss: 0.9517 - val_loss: 1.1336
Epoch 7/1000
306/306 [==============================] - 8s 25ms/step - loss: 0.9516 - val_loss: 1.1891
Epoch 8/1000
306/306 [==============================] - 8s 26ms/step - loss: 0.9518 - val_loss: 1.1723
Epoch 9/1000
306/306 [==============================] - 8s 27ms/step - loss: 0.9517 - val_loss: 1.1603
Epoch 10/1000
306/306 [==============================] - 9s 28ms/step -

[I 2024-02-17 13:34:34,749] Trial 1 finished with value: 0.16183478865132997 and parameters: {'num_layers': 1, 'num_nerons': 160, 'alpha': 0.03931274433278467, 'iter': 1000, 'b_s': 256}. Best is trial 0 with value: 0.1565835162874604.


...

[I 2024-02-17 16:43:57,709] Trial 97 finished with value: 0.061376640881331 and parameters: {'num_layers': 2, 'num_nerons': 64, 'alpha': 0.001218715687772621, 'iter': 1000, 'b_s': 576}. Best is trial 21 with value: 0.059164242877404766.


Epoch 1/1000
164/164 [==============================] - 15s 33ms/step - loss: 0.2564 - val_loss: 0.2668
Epoch 2/1000
164/164 [==============================] - 3s 19ms/step - loss: 0.1766 - val_loss: 0.2473
Epoch 3/1000
164/164 [==============================] - 3s 19ms/step - loss: 0.1594 - val_loss: 0.2471
Epoch 4/1000
164/164 [==============================] - 3s 20ms/step - loss: 0.1481 - val_loss: 0.2401
Epoch 5/1000
164/164 [==============================] - 3s 20ms/step - loss: 0.1365 - val_loss: 0.2365
Epoch 6/1000
164/164 [==============================] - 3s 19ms/step - loss: 0.1287 - val_loss: 0.2426
Epoch 7/1000
164/164 [==============================] - 3s 19ms/step - loss: 0.1237 - val_loss: 0.2375
Epoch 8/1000
164/164 [==============================] - 3s 20ms/step - loss: 0.1191 - val_loss: 0.2358
Epoch 9/1000
164/164 [==============================] - 4s 22ms/step - loss: 0.1136 - val_loss: 0.2434
Epoch 10/1000
164/164 [==============================] - 3s 19ms/step - 

[I 2024-02-17 16:45:17,583] Trial 98 finished with value: 0.06679196557256073 and parameters: {'num_layers': 2, 'num_nerons': 64, 'alpha': 0.0057172779068127375, 'iter': 1000, 'b_s': 480}. Best is trial 21 with value: 0.059164242877404766.


Epoch 1/1000
144/144 [==============================] - 14s 30ms/step - loss: 0.3554 - val_loss: 0.2662
Epoch 2/1000
144/144 [==============================] - 2s 15ms/step - loss: 0.1833 - val_loss: 0.2428
Epoch 3/1000
144/144 [==============================] - 2s 16ms/step - loss: 0.1610 - val_loss: 0.2314
Epoch 4/1000
144/144 [==============================] - 2s 15ms/step - loss: 0.1486 - val_loss: 0.2307
Epoch 5/1000
144/144 [==============================] - 2s 16ms/step - loss: 0.1400 - val_loss: 0.2243
Epoch 6/1000
144/144 [==============================] - 2s 16ms/step - loss: 0.1353 - val_loss: 0.2269
Epoch 7/1000
144/144 [==============================] - 2s 16ms/step - loss: 0.1290 - val_loss: 0.2248
Epoch 8/1000
144/144 [==============================] - 2s 16ms/step - loss: 0.1258 - val_loss: 0.2247
Epoch 9/1000
144/144 [==============================] - 2s 17ms/step - loss: 0.1231 - val_loss: 0.2237
Epoch 10/1000
144/144 [==============================] - 2s 16ms/step - 

[I 2024-02-17 16:46:37,504] Trial 99 finished with value: 0.062259895007136384 and parameters: {'num_layers': 2, 'num_nerons': 32, 'alpha': 0.003142750935346444, 'iter': 1000, 'b_s': 544}. Best is trial 21 with value: 0.059164242877404766.


In [17]:
# Trial 21 finished with value: 0.059164242877404766 and parameters: {'num_layers': 2, 'num_nerons': 32, 'alpha': 0.001324680528871899, 'iter': 1000, 'b_s': 672}.
# Input layer
model=Sequential()
#model.add(Input(X_fit.shape[0],X_fit.shape[1],X_fit.shape[2]))
# Hidden Layers
for i in range(2):
    if i==0:
        model.add(LSTM(units=32,return_sequences=True,input_shape=(X_fit.shape[1],X_fit.shape[2])))
    elif i<1:
        model.add(LSTM(units=32,return_sequences=True))
    else :
        model.add(LSTM(units=32))
# output layer
model.add(Dense(units=24))
# Training
model.compile(optimizer=Adam(learning_rate=0.001324680528871899),loss='mse')
model.fit(X_fit,y_fit,epochs=17,batch_size=672)

Epoch 1/17
117/117 [==============================] - 8s 12ms/step - loss: 0.5202
Epoch 2/17
117/117 [==============================] - 1s 12ms/step - loss: 0.2019
Epoch 3/17
117/117 [==============================] - 1s 9ms/step - loss: 0.1694
Epoch 4/17
117/117 [==============================] - 1s 10ms/step - loss: 0.1571
Epoch 5/17
117/117 [==============================] - 1s 9ms/step - loss: 0.1487
Epoch 6/17
117/117 [==============================] - 1s 10ms/step - loss: 0.1430
Epoch 7/17
117/117 [==============================] - 1s 11ms/step - loss: 0.1373
Epoch 8/17
117/117 [==============================] - 1s 11ms/step - loss: 0.1334
Epoch 9/17
117/117 [==============================] - 1s 13ms/step - loss: 0.1300
Epoch 10/17
117/117 [==============================] - 1s 11ms/step - loss: 0.1263
Epoch 11/17
117/117 [==============================] - 1s 12ms/step - loss: 0.1247
Epoch 12/17
117/117 [==============================] - 1s 13ms/step - loss: 0.1215
Epoch 13/17
117

In [18]:
model.summary()

Model: "sequential_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_212 (LSTM)             (None, 1, 32)             90240     
                                                                 
 lstm_213 (LSTM)             (None, 32)                8320      
                                                                 
 dense_101 (Dense)           (None, 24)                792       
                                                                 
Total params: 99,352
Trainable params: 99,352
Non-trainable params: 0
_________________________________________________________________


In [19]:
pred=model.predict(X_test)

12/12 [==============================] - 1s 4ms/step


In [24]:
y_test2=scaler.inverse_transform(np.array(y_test).reshape(-1,1))
pred2=scaler.inverse_transform(pred.reshape(-1,1))
error=mean_absolute_percentage_error(y_test2,pred2)
error

0.061009761438176116

In [23]:
pred.reshape(-1)

array([-0.06203908, -0.48918572, -0.8431752 , ...,  0.44775122,
        0.16909096, -0.11200303], dtype=float32)

In [25]:
pred2

array([[15180.42 ],
       [14133.217],
       [13317.456],
       ...,
       [16526.254],
       [15777.232],
       [15054.246]], dtype=float32)

In [26]:
pred2=pred.reshape(-1,1)
pred2=scaler.inverse_transform(pred2)
pred2=pd.Series(pred2.reshape(-1))

actual=df.loc[df['date_time']>='2017-08-03 00:00:00','aep_mw']
actual.index=range(len(actual))
actual.drop(8760,inplace=True)
pred_df=pd.concat([actual,pred2],axis=1)
pred_df.columns=['aep_mw','forecast']
pred_df.index=df.loc[df['date_time']>='2017-08-03 00:00:00']['date_time'].iloc[:8760]

In [27]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(actual,pred2)

0.06162556958443764

In [28]:
import plotly.graph_objects as go
figure=go.Figure()
figure.add_trace(go.Scatter(x=df['date_time'].tail(len(pred2)),y=pred_df['aep_mw'],name='actual_electricity_load'))
figure.add_trace(go.Scatter(x=df['date_time'].tail(len(pred2)),y=pred_df['forecast'],name='mlp_forecast'))